In [221]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
import importlib
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
import optuna

%matplotlib inline

In [222]:
df = pd.read_csv("train_meanprice.csv")
df_test = pd.read_csv("test_meanprice.csv")


In [223]:
df_train, df_val = train_test_split(df, test_size=0.2, shuffle=True, random_state=123)
    
col = "n"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

stacking_answer= df_val[col].to_list()

categorical_features = [
                    "Month",
                    "shopID",
                    "Cat",
                    "itemID"
                    ]

In [224]:
# OptunaによるlightGBMの最適パラメータ探索

def objective(trial):

    lgb_trains = lgb.Dataset(train_x, train_y)
    lgb_valids = lgb.Dataset(val_x, val_y)
    
    learning_rate = trial.suggest_float('learning_rate', 0.0, 1.0)
    num_leaves =  trial.suggest_int("num_leaves", 2, 100)
    tree_learner = trial.suggest_categorical('tree_learner', ["serial", "feature", "data", "voting"])
    lambda_l1= trial.suggest_float("lambda_l1", 0.0, 200.0)
    lambda_l2 = trial.suggest_float("lambda_l2", 0.0, 200.0)
    max_depth = trial.suggest_int("max_depth", 2, 12)
    num_iterations = trial.suggest_int("num_iterations", 10, 100)

    lgb_params = {
        "objective": "regression",
        "boosting_type": "gbdt", 
        "metrics": "rmse", 
        "learning_rate": learning_rate, 
        "num_leaves": num_leaves, 
        "tree_learner": tree_learner,
        "lambda_l1": lambda_l1, 
        "lambda_l2": lambda_l2, 
        "seed": 123, 
        "max_depth": max_depth,
        "num_iterations": num_iterations      
    }

    regressor = lgb.train(lgb_params,
                        lgb_trains,
                        valid_sets=lgb_valids, 
                        categorical_feature=categorical_features,
                        )

    lgb_pred_val_y = regressor.predict(val_x)

    rmse = np.sqrt(mse(val_y, lgb_pred_val_y))
    
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2023-09-08 02:05:00,955] A new study created in memory with name: no-name-32e60f72-4894-437a-ac74-50bae5323222
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000616 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 708
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 6
[LightGBM] [Info] Start training from score 0.750019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

[I 2023-09-08 02:05:01,642] Trial 0 finished with value: 1.1089270954180763 and parameters: {'learning_rate': 0.3126549807056239, 'num_leaves': 17, 'tree_learner': 'serial', 'lambda_l1': 33.60604682253019, 'lambda_l2': 178.260603740155, 'max_depth': 4, 'num_iterations': 77}. Best is trial 0 with value: 1.1089270954180763.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000270 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 708
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 6
[LightGBM] [Info] Start training from score 0.750019


[I 2023-09-08 02:05:02,304] Trial 1 finished with value: 1.0801705115714901 and parameters: {'learning_rate': 0.44213207955309086, 'num_leaves': 13, 'tree_learner': 'voting', 'lambda_l1': 67.8068157630318, 'lambda_l2': 104.68981949555165, 'max_depth': 12, 'num_iterations': 68}. Best is trial 1 with value: 1.0801705115714901.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 708
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 6
[LightGBM] [Info] Start training from score 0.750019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2023-09-08 02:06:19,973] Trial 45 finished with value: 1.018543287507909 and parameters: {'learning_rate': 0.47572629407604855, 'num_leaves': 78, 'tree_learner': 'feature', 'lambda_l1': 31.068471756825353, 'lambda_l2': 9.467159587320985, 'max_depth': 11, 'num_iterations': 60}. Best is trial 44 with value: 1.000322848408787.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000300 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 708
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 6
[LightGBM] [Info] Start training from score 0.750019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2023-09-08 02:06:22,676] Trial 46 finished with value: 1.0203174382069704 and parameters: {'learning_rate': 0.4713394401516955, 'num_leaves': 78, 'tree_learner': 'feature', 'lambda_l1': 23.06521029453424, 'lambda_l2': 10.23714963633683, 'max_depth': 11, 'num_iterations': 61}. Best is trial 44 with value: 1.000322848408787.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000296 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 708
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 6
[LightGBM] [Info] Start training from score 0.750019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

In [ ]:
# パラメータ最適化したlightGBMによる予想

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

model_lgb = lgb.train(study.best_trial.params, 
                    trains, 
                    valid_sets=valids, 
                    categorical_feature=categorical_features
                    )

val_lgb = model_lgb.predict(val_x)
rmse = np.sqrt(mse(val_y, val_lgb))

lgb_pred = model_lgb.predict(df_test)

rmse

/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000657 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 708
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 6
[LightGBM] [Info] Start training from score 0.750019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


1.0090303790797275

In [ ]:
#Catboostによる予想

cb_train = cb.Pool(train_x, label=train_y, cat_features=categorical_features)
cb_test = cb.Pool(val_x, label=val_y, cat_features=categorical_features)

model_cb = cb.CatBoostRegressor(iterations=1000, loss_function='RMSE', early_stopping_rounds=10)
model_cb.fit(cb_train)

val_cb = model_cb.predict(val_x)
rmse = np.sqrt(mse(val_y, val_cb))

cb_pred = model_cb.predict(df_test)

rmse


Learning rate set to 0.076863
0:	learn: 1.6799697	total: 6.83ms	remaining: 6.82s
1:	learn: 1.6580520	total: 11.4ms	remaining: 5.67s
2:	learn: 1.6378581	total: 17ms	remaining: 5.64s
3:	learn: 1.6202102	total: 21.2ms	remaining: 5.29s
4:	learn: 1.6071519	total: 26.7ms	remaining: 5.32s
5:	learn: 1.5924633	total: 30.9ms	remaining: 5.13s
6:	learn: 1.5800664	total: 35.2ms	remaining: 4.99s
7:	learn: 1.5637706	total: 41.3ms	remaining: 5.12s
8:	learn: 1.5530170	total: 46.2ms	remaining: 5.09s
9:	learn: 1.5396529	total: 53.1ms	remaining: 5.25s
10:	learn: 1.5327906	total: 59.1ms	remaining: 5.31s
11:	learn: 1.5241970	total: 65.6ms	remaining: 5.4s
12:	learn: 1.5160439	total: 70.7ms	remaining: 5.36s
13:	learn: 1.5100778	total: 76ms	remaining: 5.35s
14:	learn: 1.5049834	total: 80.9ms	remaining: 5.32s
15:	learn: 1.4997763	total: 86ms	remaining: 5.29s
16:	learn: 1.4945037	total: 90.2ms	remaining: 5.22s
17:	learn: 1.4868719	total: 96.6ms	remaining: 5.27s
18:	learn: 1.4835268	total: 102ms	remaining: 5.24s


1.0265223528439864

In [ ]:
# XGBoostによる予想

xgb_train = xgb.DMatrix(train_x, label=train_y)
xgb_val = xgb.DMatrix(val_x, label=val_y)
xgb_test = xgb.DMatrix(df_test)

params = {'objective':'reg:squarederror',
          'random_state':123}

model_xgb = xgb.train(params, dtrain=xgb_train, num_boost_round=1000)

val_xgb = model_xgb.predict(xgb_val)
rmse = np.sqrt(mse(val_y, val_xgb))

xgb_pred = model_xgb.predict(xgb_test)

rmse


0.981074503560952

In [ ]:
stacking_list = []
stacking_list.append(val_lgb.tolist())
stacking_list.append(val_cb.tolist())
stacking_list.append(val_xgb.tolist())
stacking_list.append(stacking_answer)

stacking_df_t = pd.DataFrame(stacking_list, index=["lgb","cb", "xgb", "answer"])

stacking_df = stacking_df_t.T

In [ ]:
st_train, st_val = train_test_split(stacking_df, test_size=0.2, shuffle=True, random_state=123)

st_train_y = st_train["answer"]
st_train_x = st_train.drop("answer", axis=1)

st_val_y = st_val["answer"]
st_val_x = st_val.drop("answer", axis=1)


In [ ]:
def objective(trial):

    st_lgb_trains = lgb.Dataset(st_train_x, st_train_y)
    st_lgb_valids = lgb.Dataset(st_val_x, st_val_y)
    
    learning_rate = trial.suggest_float('learning_rate', 0.0, 1.0)
    num_leaves =  trial.suggest_int("num_leaves", 2, 100)
    tree_learner = trial.suggest_categorical('tree_learner', ["serial", "feature", "data", "voting"])
    lambda_l1= trial.suggest_float("lambda_l1", 0.0, 200.0)
    lambda_l2 = trial.suggest_float("lambda_l2", 0.0, 200.0)
    max_depth = trial.suggest_int("max_depth", 2, 12)
    num_iterations = trial.suggest_int("num_iterations", 10, 100)

    lgb_params = {
        "objective": "regression",
        "boosting_type": "gbdt", 
        "metrics": "rmse", 
        "learning_rate": learning_rate, 
        "num_leaves": num_leaves, 
        "tree_learner": tree_learner,
        "lambda_l1": lambda_l1, 
        "lambda_l2": lambda_l2, 
        "seed": 123, 
        "max_depth": max_depth,
        "num_iterations": num_iterations      
    }

    regressor = lgb.train(lgb_params,
                        st_lgb_trains,
                        valid_sets=st_lgb_valids
                        )

    lgb_pred_st_val_y = regressor.predict(st_val_x)

    rmse = np.sqrt(mse(st_val_y, lgb_pred_st_val_y))
    
    return rmse

st_study = optuna.create_study(direction='minimize')
st_study.optimize(objective, n_trials=100)

[I 2023-09-08 01:42:06,872] A new study created in memory with name: no-name-501fc8bc-df94-4dc1-bed2-cb2c64cbb570
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-09-08 01:42:06,966] Trial 0 finished with value: 0.9335657201823552 and parameters: {'learning_rate': 0.6377706171716899, 'num_leaves': 64, 'tree_learner': 'data', 'lambda_l1': 80.69614184261337, 'lambda_l2': 4.337979805915815, 'max_depth': 7, 'num_iterations': 70}. Best is trial 0 with value: 0.9335657201823552.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-09-08 01:42:06,995] Trial 1 finished with

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010874 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 10771, number of used features: 3
[LightGBM] [Info] Start training from score 0.747099
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-09-08 01:42:07,144] Trial 4 finished with value: 0.9160688052040487 and parameters: {'learning_rate': 0.8062307982236994, 'num_leaves': 43, 'tree_learner': 'serial', 'lambda_l1': 120.28701623793638, 'lambda_l2': 199.72118591440577, 'max_depth': 3, 'num_iterations': 75}. Best is trial 1 with value: 0.8818460956787404.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-09-08 01:42:07,181] Trial 5 finished with value: 1.1987920491018436 and parameters: {'learning_rate': 0.01464431158693813, 'num_leaves': 22, 'tree_lear

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000184 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 10771, number of used features: 3
[LightGBM] [Info] Start training from score 0.747099
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2023-09-08 01:42:07,453] Trial 9 finished with value: 1.010992521786784 and parameters: {'learning_rate': 0.7079359256872423, 'num_leaves': 59, 'tree_learner': 'serial', 'lambda_l1': 8.538033980654758, 'lambda_l2': 67.09454452300643, 'max_depth': 8, 'num_iterations': 65}. Best is trial 1 with value: 0.8818460956787404.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-09-08 01:42:07,481] Trial 10 finished with value: 0.8881482982767177 and parameters: {'learning_rate': 0.5712179133333187, 'num_leaves': 76, 'tree_learner': 'voting', 'lambda_l1': 199.61371429811354, 'lambda_l2': 124.4724011371191, 'max_depth': 12, 'num_iterations': 12}. Best is trial 1 with value: 0.8818460956787404.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-09-08 01:42:12,969] Trial 96 finished with value: 0.8913882272012232 and parameters: {'learning_rate': 0.06848734117121213, 'num_leaves': 2, 'tree_learner': 'feature', 'lambda_l1': 171.91174892985842, 'lambda_l2': 171.5280432184866, 'max_depth': 2, 'num_iterations': 82}. Best is trial 85 with value: 0.8761134708255485.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-09-08 01:42:13,044] Trial 97 finished with value: 0.8913987678444516 and parameters: {'learning_rate': 0.20684246807371431, 'num_leaves': 10, 'tree_l

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000393 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 10771, number of used features: 3
[LightGBM] [Info] Start training from score 0.747099
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000460 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 10771, number of used features: 3
[LightGBM] [Info] Start training from score 0.747099
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-09-08 01:42:13,184] Trial 99 finished with value: 0.8834563534968018 and parameters: {'learning_rate': 0.16769814785962514, 'num_leaves': 4, 'tree_learner': 'feature', 'lambda_l1': 192.5256296975454, 'lambda_l2': 159.29353346420785, 'max_depth': 6, 'num_iterations': 91}. Best is trial 85 with value: 0.8761134708255485.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000260 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 10771, number of used features: 3
[LightGBM] [Info] Start training from score 0.747099
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [ ]:
st_trains = lgb.Dataset(st_train_x, st_train_y)
st_valids = lgb.Dataset(st_val_x, st_val_y)

st_model_lgb = lgb.train(st_study.best_trial.params, 
                    st_trains, 
                    valid_sets=st_valids
                    )

st_val_lgb = st_model_lgb.predict(st_val_x)
rmse = np.sqrt(mse(st_val_y, st_val_lgb))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000469 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 10771, number of used features: 3
[LightGBM] [Info] Start training from score 0.747099
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [ ]:
test_stacking_list = []
test_stacking_list.append(lgb_pred.tolist())
test_stacking_list.append(cb_pred.tolist())
test_stacking_list.append(xgb_pred.tolist())

test_stacking_df_t = pd.DataFrame(test_stacking_list, index=["lgb","cb", "xgb"])

test_stacking_df = test_stacking_df_t.T

In [ ]:
final_predict = st_model_lgb.predict(test_stacking_df)

In [ ]:
final_predict.to_csv("submit17.csv", index=True, Header=False)

3060

In [ ]:
#df_test["n"].to_csv("submit16.csv", index=True, header=False)

In [ ]:
#lightgbmのfeature_importance関数を使って特徴量重要度を見てみる
#pd.DataFrame(model_lgbm.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
itemID,545
MeanPrice,447
shopID,442
Month,402
NewFeature,336
Cat,8
